In [159]:
import numpy as np
import csv
from numpy import loadtxt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score,StratifiedKFold
from xgboost import XGBClassifier,XGBRegressor
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error
from sklearn.model_selection import cross_validate   #Additional scklearn functions
from sklearn.model_selection import learning_curve, GridSearchCV #Perforing grid search
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,precision_score
import sklearn.metrics as metrics
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder 
import pandas as pd
from sklearn import decomposition
from matplotlib import pyplot
import xgboost as xgb

##Other Regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor,RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import f1_score,confusion_matrix,classification_report
from sklearn.metrics import roc_auc_score,cohen_kappa_score,matthews_corrcoef
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Necessary imports: 
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

from linearmodels.panel import PanelOLS
import statsmodels.formula.api as sm

# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [1]:
data = pd.read_csv('C:/Users/rajat.sinha/Downloads/XXXX/mso_pivoted_v4.csv')
data.head()

NameError: name 'pd' is not defined

In [2]:
data = data.drop(['XXXX','XXXX'],axis = 1)
data = data[data['XXXX'] == 'XXXX']
data['ID']= np.arange(0, len(data))
data

NameError: name 'data' is not defined

In [3]:
t1 = data[data['XXXX']=='XXXX']
t2 = t1[t1['XXXX'] > 0 ]
#t3 = t2[t2['XXXX']=='X']
t4 = t2[t2['XXXX'] == 202010]
t4
#print(len(t4))

NameError: name 'data' is not defined

In [4]:
data['EARLIER_KEY']

NameError: name 'data' is not defined

In [5]:
data = data.set_index(['ID'])
data

NameError: name 'data' is not defined

In [6]:
feature_cols = data.columns[0:67]
print(feature_cols)


NameError: name 'data' is not defined

In [7]:
X = pd.DataFrame(data.iloc[:,15:58])
X

NameError: name 'pd' is not defined

In [8]:
y = pd.DataFrame(data.iloc[:,60:62])
y

NameError: name 'pd' is not defined

In [208]:
mae=[]
dev=[]
rs = []
acc_rs = []
pvalue = []
eq = []
resid = []
size = []
regressors = []
Xbar = []
cohorts = []
ybar = []

In [209]:
d = X
#d['OVERALL_KEY2']
f = y

In [9]:
model_names2= (data[data['X_KEYS'].notna()])['X_KEYS'].unique()
print(len(model_names2))
print(model_names2)


NameError: name 'data' is not defined

In [211]:
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

In [212]:
for cohort in (d[d['X_KEYS'].notna()])['X_KEYS'].unique():
    
    #Cohort Selection
    cohorts.append(cohort)
    X_ = d[d['X_KEYS']==cohort]
    y_ = f[f['Y_KEYS']==cohort]
    #print(X_.shape)
    #print(y_.shape)
    
    #droppping 
    X_ = X_.drop(['X_KEYS'],axis = 1) ##,'],axis = 1)
    y_ = y_.drop(['Y_KEYS'],axis = 1) 
    
    ybar.append(y_)
    Xbar.append(X_)
    size.append(X_['XXXX'].count())
    # split data into train and test sets
    seed = 11
    test_size = 0.20
    X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=test_size, random_state=seed)
    #print X_train.shape, y_train.shape
    #print X_test.shape, y_test.shape

    #feature scaling
    from sklearn.preprocessing import StandardScaler
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
 
    #import statsmodels.regression.linear_model as lm
    regressor = XGBRegressor().fit(X_train, y_train)
    
    #avg = pd.DataFrame(X_.mean(axis=0))
    #avg = avg.T

    regressors.append(regressor) 
    ypred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    rs.append(r2_score(y_test, ypred))

    kfold = KFold(n_splits=10, shuffle=True)
    kf_cv_scores = cross_val_score(XGBRegressor(), X_train, y_train, scoring='r2')
    #kf_cv_scores = cross_val_score(regressor, X_train, y_train, cv=kfold,scoring='r2' )
    #print("K-fold CV average score: %.2f %.2f" % kf_cv_scores.mean(), kf_cv_scores.std()*100)
    #print("Accuracy: %.2f%% (%.2f%%)" % (kf_cv_scores.mean()*100, kf_cv_scores.std()*100))
    #K-fold CV average score: 0.87

    acc_rs.append(kf_cv_scores.mean()*100) 
    dev.append(kf_cv_scores.std()*100)
    mae.append(np.sqrt(mean_squared_error(y_test,ypred)))
    
    #pvalue.append(pd.DataFrame(round(regressor.pvalues,2)).transpose().values)
    
    #coeff = pd.DataFrame(regressor.params.values,columns={'Coeff'})
    #columns = pd.DataFrame(X_.columns.values,columns={'Metric'})
    #equation = pd.merge(round(coeff,0),(columns),left_index=True,right_index=True)
    #equation ['Y'] = equation['Coeff'].astype(str) + " X " + equation['Metric']
    #eq.append(pd.DataFrame(equation['Y']).transpose())
    
    #resid.append(regressor.resid.mean())


[03:52:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\rajat.sinha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[03:52:18] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:52:22] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:52:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:52:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\rajat.sinha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[03:52:33] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:52:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:52:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:52:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[03:54:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:54:58] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:54:58] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:54:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\rajat.sinha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[03:54:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:54:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:54:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:55:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\rajat.sinha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[03:55:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:55:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:55:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [10]:
d={'Cohorts':model_names2,'Rsquare':rs,'Rsquare_cross_val':acc_rs,'Rows':size,'Deviation':dev} #,#'P value':pvalue} #,'Resid':resid,'MAE':mae  
df_models = pd.DataFrame(d)
df_models

#print(len(dev))
#df_models2 = df_models[df_models['Rsquare']>.6]
#df_models2

NameError: name 'model_names2' is not defined

In [11]:
model = regressors[9]
plt.figure(figsize=(14,7))
print(model.feature_importances_)
pd.DataFrame(model.feature_importances_, index=X_.columns, 
             columns=['Importance']).sort_values('Importance',ascending=False)

(pd.Series(model.feature_importances_,index=X_.columns).sort_values(ascending=False).nlargest(25).plot(kind='barh'))


NameError: name 'regressors' is not defined

In [796]:
# Simulation to get discount-ad ratio vs GMV graph in each cohort
##Inconclusive results
#zero_nine_month-v1-H-High_Rated
#zero_nine_month-non-v1-H-High_Rated


In [64]:
coh = cohorts[0]
traits = Xbar[0]
sales = ybar[0]
model =  regressors[0]

In [12]:
cohorts

NameError: name 'cohorts' is not defined

In [409]:
rid=[]
ans=[]
rat = [] #[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1]
r = []

In [410]:
#### Writing For loop for the set of RID answer ####
for i in t4.index: 
    #rat = []
    #rid = []
    #ans = []
    #print(i)
    #print(rid)
    print(t4.loc[i,'XXXX_ID'])
    for coh in pd.DataFrame(cohorts).index:
        if(t4.loc[i,'X_KEYS'] == cohorts[coh]):
            cohort = cohorts[coh]
            t = (coh)   
    #print(t)        
    model =  regressors[t] 
    cohort 
    #print(model)
    print(cohorts[t])
    traits = pd.DataFrame(t4.loc[i,['XXXX']])
    #print(traits)
    sales = pd.DataFrame(t4.loc[i,['XXXX']])
    #print(sales)
    spent1 = t4.loc[i,['XXXX']]*1.0 
    spent2 = t4.loc[i,['XXXX']] *1.0
    spent3 = t4.loc[i,['XXXX']]*1.0
    spent4 = t4.loc[i,['XXXX']]*1.0
    spent = spent1[0] +spent2[0] +spent3[0] +spent4[0]
    #print(spent) #(3555)
    ##creation of 21 rows for single RID
    traits = traits.T
    newdf = pd.DataFrame(np.repeat(traits.values,21,axis=0))
    newdf.columns = traits.columns
    #print(newdf)
    ##current Ratio
    ratio = traits['XXXX'] / spent
    ratio
    #print(ratio)
    #Dropping 
    newdf = newdf.drop(['XXXX','XXXX'],axis = 1)
    newdf
    for t in newdf.index:
        #print(t)
        #spent = 100
        newdf.loc[t,'XXXX'] = spent-(spent*(t*.05))
        newdf.loc[t,'XXXX'] = spent*(t*.05)
        rid.append(t4.loc[i,'XXXX_ID'])
        rat.append(pd.DataFrame({t*0.05})[0])
    #print(newdf.columns)   
    newdf = newdf.drop(['X_KEYS'],axis = 1)
    #feature scaling
    #from sklearn.preprocessing import StandardScaler
    sc_X = StandardScaler()
    newdf = sc_X.fit_transform(newdf)
    print('R')
    #Prediction
    ypred = model.predict(newdf)
    #print(ypred)
    for z in ypred:
        ans.append(z)
    #r.append(rat)
    #print(ans)    
    #print(rat)
    #print(rid)
    #print(rid)
    #d={'RID':rid,'GMV':ans,'Ratio':rat} #,#'P value':pvalue} #,'Resid':resid,'MAE':mae  
    #print(d)
    #f.append(d)
print(len(rid))
print(len(rat))
print(len(ans))
#print(rat)
#print(r)
d={'RID':rid,'XXXX':ans,'Ratio':rat}
df_models = pd.DataFrame(d)
df_models
df_models.to_csv("automated_answer_mso2.csv")


69488
greater_than_nine-v1-L-Low_Rated-H
R
101290
greater_than_nine-v1-L-High_Rated-H
R
105064
greater_than_nine-v1-L-High_Rated-H
R
122838
greater_than_nine-v1-L-High_Rated-H
R
123701
greater_than_nine-v1-L-Low_Rated-H
R
124288
greater_than_nine-v1-L-Low_Rated-H
R
124613
greater_than_nine-v1-L-Low_Rated-H
R
129613
greater_than_nine-v1-L-Low_Rated-H
R
129637
greater_than_nine-v1-L-High_Rated-H
R
129638
greater_than_nine-v1-L-High_Rated-H
R
146821
greater_than_nine-v1-L-Low_Rated-H
R
154725
greater_than_nine-v1-L-High_Rated-H
R
159790
greater_than_nine-v1-L-Low_Rated-H
R
161753
greater_than_nine-v1-L-High_Rated-H
R
162205
greater_than_nine-v1-L-High_Rated-H
R
165181
greater_than_nine-v1-L-Low_Rated-H
R
167905
greater_than_nine-v1-L-High_Rated-H
R
172783
greater_than_nine-v1-L-Low_Rated-H
R
188029
greater_than_nine-v1-L-Low_Rated-H
R
192976
greater_than_nine-v1-H-Low_Rated-H
R
197519
greater_than_nine-v1-L-Low_Rated-H
R
197999
greater_than_nine-v1-L-Low_Rated-H
R
198006
greater_than_nine

PermissionError: [Errno 13] Permission denied: 'automated_answer_mso.csv'

In [411]:
# model3= (data[data[['City_key','key']].notna()])[['City_key']].unique()
# print(len(model3))
# print(model3)
df_models.to_csv("automated_answer_mso2.csv")


In [13]:
t = data[['XXXX','XXXX_ID']]
t

NameError: name 'data' is not defined

In [14]:
traits 

NameError: name 'traits' is not defined

In [89]:
sales.mean(axis=0)

GMV    455936.001137
dtype: float64

In [90]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [15]:
avg = pd.DataFrame(traits.mean(axis=0))
avg = avg.T
print(avg)
#maxs = X_.max(axis=0)
spent = avg['XXXX']+avg['XXXX']+avg['XXXX']+avg['XXXX']
spent #(3555)
#avg = avg.drop(['ads_only_ADS_SPENDS','disc_only_DISCOUNT'],axis = 1)
#i = 0 

NameError: name 'pd' is not defined

In [16]:
newdf = pd.DataFrame(np.repeat(avg.values,21,axis=0))
newdf.columns = avg.columns
newdf

NameError: name 'pd' is not defined

In [17]:
ratio = avg['XXXX'] / spent[0]
ratio

NameError: name 'avg' is not defined

In [18]:
#newdf.loc[1,'XXXX']
newdf = newdf.drop(['XXXX','XXXX'],axis = 1)
newdf

NameError: name 'newdf' is not defined

In [19]:
#newdf['XXXX'] = newdf.apply (lambda x: spent*(x*0.05), axis=1)
spent[0]

NameError: name 'spent' is not defined

In [124]:
for i in newdf.index:
    #print(i)
    #spent = 100
    newdf.loc[i,'XXXX'] = spent[0]-(spent[0]*(i*.05))
    newdf.loc[i,'XXXX'] = spent[0]*(i*.05)
    
    

In [679]:
#newdata.dtype()
#newdf
#X_
#newdf.to_csv("newdf_mso.csv")

In [125]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
newdf = sc_X.fit_transform(newdf)


In [126]:
ypred = model.predict(newdf)

In [127]:
t = pd.DataFrame(ypred)
t.to_csv("cohort_fifteen_mso.csv")

In [ ]:
#Preparing dataset for %to be allocated for better GMV

In [20]:
newdf2 = pd.DataFrame(np.repeat(avg.values,21,axis=0))
newdf2.columns = avg.columns
newdf2

NameError: name 'pd' is not defined

In [21]:
#newdf.loc[1,'XXXX']
newdf2 = newdf2.drop(['XXXX','XXXX'],axis = 1)
newdf2

NameError: name 'newdf2' is not defined

In [138]:
ratio[0]
i=0


In [139]:
for i in newdf2.index:
    #print(i)
    #spent[0] # = 100
    #ratio
    newdf2.loc[i,'ads_only_ADS_SPENDS'] = spent[0]*(0.05*(15+i))*(1-ratio[0])
    newdf2.loc[i,'disc_only_DISCOUNT'] = spent[0]*(0.05*(15+i))*(ratio[0])

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [830]:
#newdf2 = newdf2.drop(labels = None,axis = 0,index = 20)

In [22]:
newdf2

NameError: name 'newdf2' is not defined

In [133]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
newdf2 = sc_X.fit_transform(newdf2)

In [134]:
ypred = model.predict(newdf2)

In [135]:
t = pd.DataFrame(ypred)
t.to_csv("cohort_four_output3_mso.csv")

In [23]:
avg

NameError: name 'avg' is not defined

In [ ]:
#Making Simulator at RID basis